In [12]:
import numpy as np
import keras
import json
import csv
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD
from keras import backend as K

In [13]:
def rows_of(array):
    if len(array.shape) > 1: 
        return [(n, array[n]) for n in range(array.shape[0])]
    else: return [(0, array)]

In [14]:
def write_matrix_csv(fname, matrix_gen):
    with open(fname, 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile, delimiter=',', quoting=csv.QUOTE_MINIMAL)
        for (precolumns, matrix_array) in matrix_gen:
            for (n, row_array) in rows_of(matrix_array):
                if len(matrix_array.shape) > 1: rows, columns = matrix_array.shape
                else: rows, columns = 1, matrix_array.shape[0]
                csvwriter.writerow(precolumns + [rows, columns, n] + list(row_array))

In [15]:
def layer_weights(model):
    for layer in model.layers:
        for weight_variable in layer.weights:
            weight_values = K.batch_get_value(weight_variable)
            yield ([layer.name, weight_variable.name], weight_values)            

In [16]:
model = Sequential()
model.add(Dense(5, activation='linear', input_dim = 3, kernel_initializer='ones'))
model.add(Dense(3, activation='linear', kernel_initializer='ones'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 5)                 20        
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 18        
Total params: 38
Trainable params: 38
Non-trainable params: 0
_________________________________________________________________


In [17]:
with open('test_linear_model/config.json', 'w') as modelfile:
    json.dump(model.get_config(), modelfile)
write_matrix_csv('test_linear_model/weights_pretrain.csv', layer_weights(model))

In [18]:
input = np.array([[0.,0.,0.],[0.,0.,1.],[0.,1.,0.],[1.,0.,0.]])
output = model.predict(input)
write_matrix_csv('test_linear_model/predict_pretrain.csv', [(['in'], input), (['out'], output)])

In [19]:
sgd = SGD(lr=0.001, decay=1e-8)
model.compile(loss='mean_squared_error', optimizer=sgd)

In [20]:
x_train = np.random.random(size=(100,3))
y_train = x_train

In [21]:
model.fit(x_train, y_train, epochs=100)

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
100/100 [==============================] - 0s 730us/step - loss: 52.8891
Epoch 2/100
100/100 [==============================] - 0s 50us/step - loss: 43.2425
Epoch 3/100
100/100 [==============================] - 0s 60us/step - loss: 35.4064
Epoch 4/100
100/100 [==============================] - 0s 50us/step - loss: 29.3411
Epoch 5/100
100/100 [==============================] - 0s 40us/step - loss: 25.0324
Epoch 6/100
100/100 [==============================] - 0s 50us/step - loss: 21.5414
Epoch 7/100
100/100 [==============================] - 0s 50us/step - loss: 18.9457
Epoch 8/100
100/100 [==============================] - 0s 40us/step - loss: 16.7353
Epoch 9/100
100/100 [==============================] - 0s 40us/step - loss: 14.9120
Epoch 10/100
100/100 [==============================] - 0s 50us/step - loss: 13.2481
Epoch 11/100
100/100 [==============================] - 0s 50us/step - loss: 11.5694
Epoch 12/100
100/100 [===

100/100 [==============================] - 0s 50us/step - loss: 0.7611
Epoch 96/100
100/100 [==============================] - 0s 40us/step - loss: 0.7587
Epoch 97/100
100/100 [==============================] - 0s 40us/step - loss: 0.7560
Epoch 98/100
100/100 [==============================] - 0s 50us/step - loss: 0.7530
Epoch 99/100
100/100 [==============================] - 0s 60us/step - loss: 0.7500
Epoch 100/100
100/100 [==============================] - 0s 40us/step - loss: 0.7474


In [22]:
write_matrix_csv('test_linear_model/weights_posttrain.csv', layer_weights(model))

In [23]:
input = np.array([[0.,0.,0.],[0.,0.,1.],[0.,1.,0.],[1.,0.,0.]])
output = model.predict(input)
write_matrix_csv('test_linear_model/predict_posttrain.csv', [(['in'], input), (['out'], output)])